In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('dataframe').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 10:02:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

### REading dataset

In [6]:
spark.read.option('header', 'true').csv('test1.csv').show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
df_pyspark = spark.read.option('header', 'true').csv('test1.csv')

### checking the schema
equivalent to df.info in pandas

In [8]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



to automatically get proper data types from dataset

In [9]:
df_pyspark = spark.read.option('header', 'true').csv('test1.csv', inferSchema=True)

In [10]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



can do both header and infer schema in single line if needed

In [11]:
df_pyspark = spark.read.csv('test1.csv', header=True, inferSchema=True )

In [12]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



to check the type of the dataframe.

In [13]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

to get the list of columns

In [14]:
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

equivalent to head in pandas but we get it here as list instead of df like in pandas

In [16]:
df_pyspark.head(4)

[Row(Name='Krish', age=31, Experience=10, Salary=30000),
 Row(Name='Sudhanshu', age=30, Experience=8, Salary=25000),
 Row(Name='Sunny', age=29, Experience=4, Salary=20000),
 Row(Name='Paul', age=24, Experience=3, Salary=20000)]

to select a single colums. the return type is dataframe

In [17]:
df_pyspark.select('Name')

DataFrame[Name: string]

In [19]:
type(df_pyspark.select('Name'))

pyspark.sql.dataframe.DataFrame

.show shows the column

In [18]:
df_pyspark.select('Name').show()

+---------+
|     Name|
+---------+
|    Krish|
|Sudhanshu|
|    Sunny|
|     Paul|
|   Harsha|
|  Shubham|
+---------+



to pickup multiple columns, we have to give the list of columns

In [21]:
df_pyspark.select(['Name', 'Experience']) #.show()

DataFrame[Name: string, Experience: int]

to check the datatypes of the columns

In [22]:
df_pyspark.dtypes

[('Name', 'string'), ('age', 'int'), ('Experience', 'int'), ('Salary', 'int')]